
**Definition - Difference equation**

Consider a sequence of numbers \\(u_0, u_1, u_2...\\) which is related by the equation 

$$u_{n} - \alpha u_{n-1} = 0$$

Where \\(\alpha\\) is not 0 and \\(n=1,2,..\\)

This is a first order difference equation

To solve a difference equation we need an initial condition \\(u_0=c\\) to write

$$u_1 = \alpha u_2$$

And iterating all the way up to \\(u_n\\)

$$u_n = \alpha^n u_0 = \alpha^n c$$

The difference equation can be rewritten in operator notation

$$u_n - \alpha u_{n-1} = (1-\alpha B ) u_n$$

Here the polynomial for the equation is \\( \alpha(z) = 1 - \alpha z\\) with the first root \\(z_0=\frac{1}{\alpha}\\).

The root can be inserted into \\(u_n = \alpha^n c\\) to write a solution in terms of the first root \\(z_0\\).

$$u_n = (z_0^{-1})^n c$$

Here it is seen that a solution to a first order difference equation dependes entirely on the initial condition and the inverse of the root.



**Definition - General solutions to difference equations**

From Time Series Analysis and Its Applications, Shumway & Stoffer page 98 it can be seen that general solutions to difference equations can be obtained.

For a second order difference equation, if \\(z_0\\) and \\(z_1\\) are not the same the general solution is

$$u_n  = c_0 z_0^{-n} + c_0 z_1^{-n}$$

When \\(z_0=z_1\\) a general solution will be 

$$u_n  = z_0^{-n}(c_0 + c_1 n)$$

For these real cases, where the system is causal(\\(|z|>1\\)) the value of \\(u_n\\) will decrease exponentially.

When the roots are complex we will have a conjugate pair, since the value are real.

\\(z_0 = \alpha + \beta j\\) and \\(z_1 = \alpha - \beta j\\)

And the general solution will be 

$$ u_n = |z|^n(c_1 cos(n \theta)+c_2 sin(n \theta)) $$

Where \\(c_1\\) and \\(c_2\\) are constants determined by initial conditions and \\(\theta=tan^{-1}(\beta/\alpha)\\)

For these complex case, where the system is causal(\\(|z|>1\\)) the value of \\(u_n\\) will be a sinusoid with amplitude decreasing exponentially.


**Example - ACF of AR(2) using difference equations**

Given a **AR**(2) process 

$$x_t = \phi_1 x_{t-1} + \phi_2 x_{t-2} + w_t$$

We can find the autocovariance by multiplying each side with \\(x_{t-h}\\) and taking the expectation

$$E[x_t x_{t-h}] = \phi_1 E[x_{t-1} x_{t-h}] + \phi_2 E[x_{t-2} x_{t-h}] + E[w_t x_{t-h}]$$

Since \\(\gamma(h)=E[x_t x_{t-h}]\\) and \\(E[w_t x_{t-h}] = 0\\) we write

$$\gamma(h) = \phi_1 \gamma(h-1) + \phi_2 \gamma(h-2)$$

To obtain the ACF, we divide through on both sides with \\(\gamma(0)\\)

$$\rho(h) = \phi_1 \rho(h-1) + \phi_2 \rho(h-2)$$

By definition the ACF has \\(\rho(0)=1\\) since \\(\rho(h)=\frac{\lambda(h)}{\lambda(0)}\\) and \\(\rho(-1) = \rho(1)\\)

When \\(h=1\\) the other initial condition(\\(\rho(0)=1\\) being the first) can be found as 

$$\rho(1) - \phi_1 \rho(0) - \phi_2 \rho(-1) = \rho(1) - \phi_1 - \phi_2 \rho(1) = \rho(1) (1 - \phi_2) - \phi_1 = 0 $$

$$ \rho(1) = \frac{\phi_1}{(1 - \phi_2)}$$

When \\(h>1\\) the value can be found by using the difference equation itself and the previous values.


In [0]:
## Example ACF of AR(2) process with real roots
# (1-0.4z)(1-0.3z) = 1 - 0.7z + 0.12z^2
# phi_1 = 0.7, phi_2 = 0.12

import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf

N = 100000

mu, sigma = 0, 1
phi_1, phi_2 = 0.7, 0.12
w = np.random.normal(mu, sigma, N)
x = np.zeros(N)

for i in range(2, N):
    x[i] = phi_1 * x[i-1] + phi_2 * x[i-2] + w[i]

# Theoretical values of autocorrelation according to previous example
def theoretical_acf(lag, phi_1, phi_2):
    if lag == 0:
        return 1
    elif lag == 1:
        return phi_1/(1-phi_2)
    else:
        return phi_1 * theoretical_acf(lag-1, phi_1, phi_2) + phi_2 * theoretical_acf(lag-2, phi_1, phi_2)

plot_lags = 20

plt.figure()
plt.plot(acf(x, nlags=plot_lags), "-*", label="Calculated value")
plt.scatter([lag for lag in range(plot_lags+1)],[theoretical_acf(lag, phi_1, phi_2) for lag in range(plot_lags+1)], color="red", label="Theoretical value")
plt.title(f"Autocorrelation of AR(2)")
plt.xlabel("Lag")
plt.ylabel("Value")
plt.legend()

**Example - AR(2) process with complex roots**

Consider the **AR**(2) process \\(x_t=1.5x_{t-1}-0.75x_{t-2}+w_t\\)

This has the charateristic equation \\(\phi(z)=1-1.5z+0.75z^2\\) 

Solving this yields the roots \\(z_0=1 - 0.57735j\\) and \\(z_1=1 + 0.57735j\\)

Finding the argument \\(\theta = tan^{-1}(0.57735/1)=0.523599 [rad]\\)

Since a sinusoid repeats every \\(2\pi\\) we will have roughly 12 samples between the cycles repeat. This can be seen in the ACF below.

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf

N = 10000

mu, sigma = 0, 1
phi_1, phi_2 = 1.5, -0.75
w = np.random.normal(mu, sigma, N)
x = np.zeros(N)

for i in range(2, N):
    x[i] = phi_1 * x[i-1] + phi_2 * x[i-2] + w[i]

plt.figure()
plt.plot(x)
plt.title(f"Realization of AR(2)")
plt.xlabel("Sample")
plt.ylabel("Value")

plt.figure()
plt.plot(acf(x, nlags=plot_lags), "-*", label="Calculated value")
plt.scatter([lag for lag in range(plot_lags+1)],[theoretical_acf(lag, phi_1, phi_2) for lag in range(plot_lags+1)], color="red", label="Theoretical value")
plt.title(f"Autocorrelation of AR(2) with complex roots")
plt.xlabel("Lag")
plt.ylabel("Value")
plt.legend()